# Compare QTLs for tomato fruit shape & potato tuber shape

## Background

Reference tomatoes have a round fruit shape while the reference potatoes have an elongated tuber shape. Recently published article (<a href="https://dx.doi.org/10.1038%2Fs41467-018-07216-8">Wu et al., 2018</a>) reveals that the SLOFP20 gene present on the tomato chromosome 10 is responsible for round fruits. However, this gene does not have an ortholog in the reference potato genome (DM), which results in elongated tuber. This notebook is used to map the QTL regions in both tomato and potato. First, genes in the QTLs are classified into three categories: i) corresponding genes (orthologs) between tomato and potato, ii) tomato genes with no orthologs in potato and iii) vice versa. For each category, GO annotations are retrieved and then compared among the categories. 

## Initialization

In [1]:
import requests
import io
import pandas as pd
import numpy as np
#from diagrams import DIAGRAMS
from IPython.display import Image,SVG
from itertools import chain

In [2]:
url = dict(local='http://localhost:8088/api/local/local',
           remote='http://pbg-ld.candygene-nlesc.surf-hosted.nl:8088/api/candYgene/queries')
base_url = url['remote']
headers = {'accept': 'text/csv'}
#print(base_url)

In [3]:
# genes delineating QTLs for (tomato) fruit and (potato) tuber shapes
qtl_genes = dict(tomato=['Solyc10g075170.1', 'Solyc10g076240.1'],
                 potato=['PGSC0003DMG400006678', 'PGSC0003DMG400020801'])

In [4]:
# retrieve genomic locations for the genes
genes = pd.DataFrame()
for g in qtl_genes['tomato']: #list(chain(*(qtl_genes.values()))):
    try:
        if g is not None:
            with requests.get(base_url + '/getFeatureLocation',
                              params = {'featureid': "'%s'" % g},
                              headers = headers) as req:
                genes = genes.append(pd.read_csv(io.StringIO(req.text)), ignore_index=True)
    except:
        print('Failed to connect to the Web API!')
        break

In [5]:
genes

,feature_id,feature_name,chrom,begin_pos,end_pos,taxon_id
0,Solyc10g075170.1,protein_coding_gene,chromosome 10,58891402,58895882,4081
1,Solyc10g076240.1,protein_coding_gene,chromosome 10,59082990,59084119,4081


In [6]:
#d = genes.groupby(['taxon_id', 'chrom']).apply(lambda x: np.sort(np.array(list(chain(*[x.begin_pos, x.end_pos])))))
#t = d[4081]['chromosome 10']
#p = d[4113]['chromosome 10']
#print(t)
#print(p)
#print(t[0], t[-1])
#print(p[0], p[-1])

In [7]:
# compute QTL interval given the start/end positions of the delineating genes
pos = pd.concat([genes['begin_pos'], genes['end_pos']]).describe()
interval = dict(chrom=genes['chrom'].unique()[0],
                taxon_id=int(genes['taxon_id'].unique()[0]),
                begin=int(pos['min']),
                end=int(pos['max']))

In [8]:
interval

{'chrom': 'chromosome 10',
 'taxon_id': 4081,
 'begin': 58891402,
 'end': 59084119}

In [9]:
# retrieve all genes (incl. their coordinates) in the interval
genes_interval = pd.DataFrame()
params = {'feature': 'protein_coding_gene',
          'chrom': interval['chrom'],
          'graph': 'http://solgenomics.net/genome/Solanum_lycopersicum',
          'begin': interval['begin'],
          'end': interval['end']}
try:
    with requests.get(base_url + '/getFeaturesInInterval',
                      params=params,
                      headers=headers) as req:
        genes_interval = pd.read_csv(io.StringIO(req.text))
except:
    print('Failed to connect to the Web API!')

In [10]:
genes_interval

,feature_id,feature_name,chrom,begin_pos,end_pos
0,Solyc10g075170.1,protein_coding_gene,chromosome 10,58891402,58895882
1,Solyc10g076170.1,protein_coding_gene,chromosome 10,58981351,58981887
2,Solyc10g076180.1,protein_coding_gene,chromosome 10,59006329,59007294
3,Solyc10g076190.1,protein_coding_gene,chromosome 10,59045991,59047132
4,Solyc10g076200.1,protein_coding_gene,chromosome 10,59051770,59052216
5,Solyc10g076210.1,protein_coding_gene,chromosome 10,59054478,59055612
6,Solyc10g076220.1,protein_coding_gene,chromosome 10,59059828,59060961
7,Solyc10g076230.1,protein_coding_gene,chromosome 10,59074041,59074647
8,Solyc10g076240.1,protein_coding_gene,chromosome 10,59082990,59084119


In [11]:
# get orthologs in potato
rows = []
for index, g in genes_interval.iterrows():
    try:
        with requests.get(base_url + '/getOrthologs',
                          params={'geneid': "'%s'" % g['feature_id']},
                          headers=headers) as req:
            df = pd.read_csv(io.StringIO(req.text))
            if df.size == 0:
                rows.append([g['feature_id'], g['chrom'], g['begin_pos'], g['end_pos'],
                             None, None, None, None])
                continue
            for index, j in df.iterrows():
                with requests.get(base_url + '/getFeatureLocation',
                    params={'featureid': "'%s'" % j['ortholog_id']},
                    headers=headers) as req:
                    for index, o in pd.read_csv(io.StringIO(req.text)).iterrows():
                        rows.append([g['feature_id'], g['chrom'], g['begin_pos'], g['end_pos'],
                                          o['feature_id'], o['chrom'], o['begin_pos'], o['end_pos']])
  
    except:
        print('Failed to connect to the Web API!')
        break

In [12]:
cols = ['gene_id', 'chrom', 'begin', 'end', 'ortho_id', 'ortho_chrom', 'ortho_begin', 'ortho_end']
orthologs = pd.DataFrame(rows, columns=cols)
orthologs['ortho_begin'] = orthologs['ortho_begin'].fillna(-1)
orthologs['ortho_end'] = orthologs['ortho_end'].fillna(-1)
orthologs['ortho_begin'] = orthologs['ortho_begin'].astype(np.int64)
orthologs['ortho_end'] = orthologs['ortho_end'].astype(np.int64)
orthologs = orthologs.replace([-1], [None])
orthologs

,gene_id,chrom,begin,end,ortho_id,ortho_chrom,ortho_begin,ortho_end
0,Solyc10g075170.1,chromosome 10,58891402,58895882,PGSC0003DMG400006678,chromosome 10,48978066,48982521
1,Solyc10g076170.1,chromosome 10,58981351,58981887,None,None,None,None
2,Solyc10g076180.1,chromosome 10,59006329,59007294,None,None,None,None
3,Solyc10g076190.1,chromosome 10,59045991,59047132,PGSC0003DMG400011948,chromosome 0,22096297,22097627
4,Solyc10g076200.1,chromosome 10,59051770,59052216,PGSC0003DMG400011955,chromosome 0,22098971,22100334
5,Solyc10g076200.1,chromosome 10,59051770,59052216,PGSC0003DMG400040954,chromosome 10,49170543,49171657
6,Solyc10g076210.1,chromosome 10,59054478,59055612,PGSC0003DMG400020799,chromosome 10,49151461,49152451
7,Solyc10g076210.1,chromosome 10,59054478,59055612,PGSC0003DMG400020800,chromosome 10,49084193,49085040
8,Solyc10g076220.1,chromosome 10,59059828,59060961,PGSC0003DMG400020799,chromosome 10,49151461,49152451
9,Solyc10g076220.1,chromosome 10,59059828,59060961,PGSC0003DMG400020800,chromosome 10,49084193,49085040


In [13]:
# show number of orthologs per gene
display(orthologs.groupby(['gene_id']).agg({'ortho_id':['nunique']}))

,ortho_id
,nunique
gene_id,
Solyc10g075170.1,1
Solyc10g076170.1,0
Solyc10g076180.1,0
Solyc10g076190.1,1
Solyc10g076200.1,2
Solyc10g076210.1,2
Solyc10g076220.1,2
Solyc10g076230.1,1


## Potato
### Find interval
Find locations for genes, and compute interval

In [14]:
try:
    resp1 = requests.get(url+"/getFeatureLocation", 
                        params={"featureid": "'"+pg1+"'"}, 
                        headers={"accept": "application/json"})
    resp2 = requests.get(url+"/getFeatureLocation", 
                        params={"featureid": "'"+pg2+"'"}, 
                        headers={"accept": "application/json"})
    intervalP = {}
    intervalP["chrom"] = resp1.json()["results"]["bindings"][0]["chrom"]["value"]
    intervalP["taxon_id"] = resp1.json()["results"]["bindings"][0]["taxon_id"]["value"]
    intervalP["begin_pos"] = int(resp1.json()["results"]["bindings"][0]["end_pos"]["value"])
    intervalP["end_pos"] = int(resp2.json()["results"]["bindings"][0]["begin_pos"]["value"])    
except:
    raise Exception("couldn't get interval for "+str(pg1)+" or "+str(pg2))
intervalP   

NameError: name 'pg1' is not defined

### Find genes and orthologs
Find genes and orthologs for interval

Find potato genes in interval

In [ ]:
try:
    if intervalP["taxon_id"]=="4113":
        graph = "http://solgenomics.net/genome/Solanum_tuberosum"
    else:
        raise Exception("unknown taxon_id "+str(intervalP["taxon_id"]))
    genes = requests.get(url+"/getFeaturesInInterval", 
                        params={"feature": "'protein_coding_gene'", "graph": graph,
                               "begin": intervalP["begin_pos"], "end": intervalP["end_pos"],
                               "chrom": intervalP["chrom"]}, 
                        headers={"accept": "application/json"})
    genesP_in_interval = []
    for gene in genes.json()["results"]["bindings"]:
        gene_id = gene["feature_id"]["value"]
        #get location
        location = requests.get(url+"/getFeatureLocation", 
                        params={"featureid": "'"+gene_id+"'"}, 
                        headers={"accept": "application/json"})
        chrom = location.json()["results"]["bindings"][0]["chrom"]["value"]
        taxon_id = location.json()["results"]["bindings"][0]["taxon_id"]["value"]
        begin_pos = int(location.json()["results"]["bindings"][0]["begin_pos"]["value"])
        end_pos = int(location.json()["results"]["bindings"][0]["end_pos"]["value"])    
        genesP_in_interval.append([gene_id, chrom, begin_pos, end_pos, taxon_id])
    genesP_in_interval = pd.DataFrame(genesP_in_interval)
    genesP_in_interval.columns = ["gene_id", "chrom", "begin_pos", "end_pos", "taxon_id"]
    genesP_in_interval = genesP_in_interval.set_index(["gene_id"])
except:
    raise Exception("couldn't get genes in interval "+str(intervalP["begin_pos"])+" - "+str(intervalP["begin_pos"])+" on "+str(intervalP["chrom"]))    
#display(genesP_in_interval)    

Find tomato orthologs for these genes

In [ ]:
try:
    ortholog_genesP_for_interval = []
    for gene_id,gene in genesP_in_interval.iterrows():
        orthologs = requests.get(url+"/getOrthologs", 
                        params={"geneid": "'"+gene_id+"'"}, 
                        headers={"accept": "application/json"})
        orthologsResult = orthologs.json()["results"]["bindings"]
        if len(orthologsResult)>0:
            for ortholog in orthologsResult:
                ortholog_id = ortholog["ortholog_id"]["value"]
                #get location
                location = requests.get(url+"/getFeatureLocation", 
                                params={"featureid": "'"+ortholog_id+"'"}, 
                                headers={"accept": "application/json"})
                chrom = location.json()["results"]["bindings"][0]["chrom"]["value"]
                taxon_id = location.json()["results"]["bindings"][0]["taxon_id"]["value"]
                begin_pos = int(location.json()["results"]["bindings"][0]["begin_pos"]["value"])
                end_pos = int(location.json()["results"]["bindings"][0]["end_pos"]["value"]) 
                ortholog_genesP_for_interval.append([gene_id, gene["chrom"],gene["begin_pos"],
                                                     gene["end_pos"],gene["taxon_id"],ortholog_id,
                                                     chrom, begin_pos, end_pos, taxon_id])
        else:
            ortholog_genesP_for_interval.append([gene_id, gene["chrom"],gene["begin_pos"],
                                                 gene["end_pos"],gene["taxon_id"], None, None,
                                                None, None, None])
    ortholog_genesP_for_interval = pd.DataFrame(ortholog_genesP_for_interval)            
    ortholog_genesP_for_interval.columns = ["gene_id", "chrom", "begin_pos", "end_pos", "taxon_id", 
                                            "ortholog_gene_id", "ortholog_chrom", "ortholog_begin_pos",
                                            "ortholog_end_pos", "ortholog_taxon_id"]
    ortholog_genesP_for_interval = ortholog_genesP_for_interval.set_index(["gene_id"])
except:
    raise Exception("couldn't get orthologs")  

Number of orthologs for each gene

In [ ]:
aggregations = { "ortholog_gene_id" : ["nunique"]}
display(ortholog_genesP_for_interval.groupby(["gene_id"]).agg(aggregations))

### List orthologs
List orthologs for the genes found in the interval

In [ ]:
index=pd.MultiIndex.from_tuples([tuple(x) for x in ortholog_genesP_for_interval[["chrom", "begin_pos", "end_pos", "ortholog_gene_id"]].to_records()], names=["gene_id", "chrom", "begin_pos", "end_pos", "ortholog_gene_id"])
display(ortholog_genesP_for_interval.set_index(index)[["ortholog_chrom", "ortholog_begin_pos", "ortholog_end_pos", "taxon_id"]])   

### Chromosomes for orthologs
Chromosomes containing the found orthologs

In [ ]:
kauraggregations = {"ortholog_begin_pos" : ["min","max"], "ortholog_end_pos" : ["min","max"]}
display(ortholog_genesP_for_interval.groupby(["ortholog_taxon_id", "ortholog_chrom"]).agg(aggregations))

### Diagram crosslinks
Create a diagram with crosslinks between found genes and orthologs

In [ ]:
#genesPdiagram = DIAGRAMS.crosslinks("potato", "tomato", intervalP, ortholog_genesP_for_interval)
#tmpFilename = "potato_"+pg1+"-"+pg2+".svg"
#genesPdiagram.write(tmpFilename, "svg")
#SVG(tmpFilename)

# Annotations
Compare GO annotatios in all three classes

Genes, orthologs for potato and tomato and annotations

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib_venn import venn2

In [ ]:
setT = genesT_in_interval.index.unique()
setP = genesP_in_interval.index.unique()
setTP = setT & ortholog_genesP_for_interval["ortholog_gene_id"].dropna().unique()
setPT = setP & ortholog_genesT_for_interval["ortholog_gene_id"].dropna().unique()
setT = list(set(setT)-set(setTP))
setP = list(set(setP)-set(setPT))
venn2(subsets = (len(setT), len(setTP), len(setP)), set_labels = ("Tomato", "Potato"))
#venn2(subsets = (1,2,3), set_labels = ("Tomato", "Potato"))
plt.show()

## Only in Tomato interval

In [ ]:
print(*setT, sep = "\n")

In [ ]:
try:
    termsT = []
    for gene_id in setT:
        annotations = requests.get(url+"/getGeneAnnotations", 
                                   params={"geneid": "'"+gene_id+"'"}, 
                                   headers={"accept": "application/json"})        
        for annotation in annotations.json()["results"]["bindings"]:
            uniprot_goa = annotation["uniprot_goa"]["value"].strip()
            if uniprot_goa=="":
                termsT.append([gene_id, None])
            else:
                for go_id in uniprot_goa.split(";"):                    
                    termsT.append([gene_id, go_id.strip()])
            break        
    termsT = pd.DataFrame(termsT)
    termsT.columns = ["gene_id", "go_id"]
    termsT = termsT.set_index(["gene_id"])
except:
    raise Exception("couldn't get terms for genes") 

In [ ]:
goT = set([go_id for go_id in termsT["go_id"] if not (go_id == None)])

In [ ]:
len(goT)

In [ ]:
print(*goT, sep="\n")

In [ ]:
termsT.set_index(["go_id"], append=True)

## Only in Potato

In [ ]:
print(*setP, sep = "\n")

In [ ]:
try:
    termsP = []
    for gene_id in setP:
        annotations = requests.get(url+"/getGeneAnnotations", 
                                   params={"geneid": "'"+gene_id+"'"}, 
                                   headers={"accept": "application/json"})        
        for annotation in annotations.json()["results"]["bindings"]:
            uniprot_goa = annotation["uniprot_goa"]["value"].strip()
            if uniprot_goa=="":
                termsP.append([gene_id, None])
            else:
                for go_id in uniprot_goa.split(";"):                    
                    termsP.append([gene_id, go_id.strip()])
            break        
    termsP = pd.DataFrame(termsP)
    termsP.columns = ["gene_id", "go_id"]
    termsP = termsP.set_index(["gene_id"])
except:
    raise Exception("couldn't get terms for genes") 

In [ ]:
goP = set([go_id for go_id in termsP["go_id"] if not (go_id == None)])

In [ ]:
len(goP)

In [ ]:
print(*goP, sep="\n")

In [ ]:
termsP.set_index(["go_id"], append=True)

## Both in Tomato and Potato

In [ ]:
print(*setTP, sep = "\n")
#print(*setPT, sep = "\n")

In [ ]:
try:
    termsTP = []
    for gene_id in setTP:
        annotations = requests.get(url+"/getGeneAnnotations", 
                                   params={"geneid": "'"+gene_id+"'"}, 
                                   headers={"accept": "application/json"})        
        for annotation in annotations.json()["results"]["bindings"]:
            uniprot_goa = annotation["uniprot_goa"]["value"].strip()
            if uniprot_goa=="":
                termsTP.append([gene_id, None])
            else:
                for go_id in uniprot_goa.split(";"):                    
                    termsTP.append([gene_id, go_id.strip()])
            break        
    termsTP = pd.DataFrame(termsTP)
    termsTP.columns = ["gene_id", "go_id"]
    termsTP = termsTP.set_index(["gene_id"])
except:
    raise Exception("couldn't get terms for genes") 

In [ ]:
print(*set([go_id for go_id in termsTP["go_id"] if not (go_id == None)]), sep="\n")

In [ ]:
termsTP.set_index(["go_id"], append=True)

## Exploring the Annotations of Gene *Solyc10g076180.1*

- GO Annotation
- PPI 
- STRING
- KEGG
- ALL Orthologs
- Species with no Orthologs

In [ ]:
try:
    ortholog_paralog_genesT_for_interval = []
    for gene_id,gene in genesT_in_interval.iterrows():
        #get paralogs, and then orthologs
        paralogs = requests.get(url+"/getParalogs", 
                        params={"geneid": "'"+gene_id+"'"}, 
                        headers={"accept": "application/json"})
        for paralog in paralogs.json()["results"]["bindings"]:
            paralog_id = paralog["paralog_id"]["value"]
            #get orthologs for paralog
            orthologs = requests.get(url+"/getOrthologs", 
                        params={"geneid": "'"+paralog_id+"'"}, 
                        headers={"accept": "application/json"})
            for ortholog in orthologs.json()["results"]["bindings"]:
                ortholog_id = ortholog["ortholog_id"]["value"]
                #get location
                location = requests.get(url+"/getFeatureLocation", 
                                params={"featureid": "'"+ortholog_id+"'"}, 
                                headers={"accept": "application/json"})
                chrom = location.json()["results"]["bindings"][0]["chrom"]["value"]
                taxon_id = location.json()["results"]["bindings"][0]["taxon_id"]["value"]
                begin_pos = int(location.json()["results"]["bindings"][0]["begin_pos"]["value"])
                end_pos = int(location.json()["results"]["bindings"][0]["end_pos"]["value"]) 
                ortholog_paralog_genesT_for_interval.append([gene_id, gene["chrom"],gene["begin_pos"],
                                                             gene["end_pos"],gene["taxon_id"],paralog_id,
                                                             "PARALOG", ortholog_id,
                                                             chrom, begin_pos, end_pos, taxon_id])
        #get direct orthologs
        orthologs = requests.get(url+"/getOrthologs", 
                        params={"geneid": "'"+gene_id+"'"}, 
                        headers={"accept": "application/json"})
        for ortholog in orthologs.json()["results"]["bindings"]:
            ortholog_id = ortholog["ortholog_id"]["value"]
            #get location
            location = requests.get(url+"/getFeatureLocation", 
                            params={"featureid": "'"+ortholog_id+"'"}, 
                            headers={"accept": "application/json"})
            chrom = location.json()["results"]["bindings"][0]["chrom"]["value"]
            taxon_id = location.json()["results"]["bindings"][0]["taxon_id"]["value"]
            begin_pos = int(location.json()["results"]["bindings"][0]["begin_pos"]["value"])
            end_pos = int(location.json()["results"]["bindings"][0]["end_pos"]["value"]) 
            ortholog_paralog_genesT_for_interval.append([gene_id, gene["chrom"],gene["begin_pos"],
                                                 gene["end_pos"],gene["taxon_id"],None, "ORTHOLOG", ortholog_id,
                                                 chrom, begin_pos, end_pos, taxon_id])    
    #create dataframe        
    ortholog_paralog_genesT_for_interval = pd.DataFrame(ortholog_paralog_genesT_for_interval)            
    ortholog_paralog_genesT_for_interval.columns = ["gene_id", "chrom", "begin_pos", "end_pos", "taxon_id", "paralog_gene_id",
                                            "path", "ortholog_gene_id", "ortholog_chrom", "ortholog_begin_pos",
                                            "ortholog_end_pos", "ortholog_taxon_id"]
    ortholog_paralog_genesT_for_interval = ortholog_paralog_genesT_for_interval.set_index(["gene_id"])
except:
    raise Exception("couldn't get orthologs")  

Number of orthologs for each gene

In [ ]:
aggregations = { "ortholog_gene_id" : ["nunique"]}
display(ortholog_paralog_genesT_for_interval.groupby(["gene_id"]).agg(aggregations))

In [ ]:
index=pd.MultiIndex.from_tuples([tuple(x) for x in ortholog_paralog_genesT_for_interval[["chrom", "begin_pos", "end_pos", "paralog_gene_id", "ortholog_gene_id"]].to_records()], names=["gene_id", "chrom", "begin_pos", "end_pos", "paralog_gene_id", "ortholog_gene_id"])
display(ortholog_paralog_genesT_for_interval.set_index(index)[["path", "ortholog_chrom", "ortholog_begin_pos", "ortholog_end_pos"]])      